In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
from pyvbmc import VBMC
import corner
from psiam_tied_dv_map_utils import rho_A_t_fn, up_RTs_fit_fn, down_RTs_fit_fn, up_RTs_fit_single_t_fn, psiam_tied_data_gen_wrapper, psiam_tied_data_gen_wrapper_V2
import sys
import multiprocessing
from psiam_tied_no_dv_map_utils import cum_A_t_fn
from psiam_tied_no_dv_map_utils import rho_A_t_fn, cum_A_t_fn, rho_E_minus_small_t_NORM_fn
from psiam_tied_dv_map_utils import cum_E_t_fn
from tqdm import tqdm
from scipy.integrate import trapezoid
import random


In [4]:
# read out_LED.csv as dataframe
og_df = pd.read_csv('../out_LED.csv')

# chose non repeat trials - 0 or 2 or missing
df = og_df[ og_df['repeat_trial'].isin([0,2]) | og_df['repeat_trial'].isna() ]

# only session type 7
session_type = 7    
df = df[ df['session_type'].isin([session_type]) ]

# training level 16
training_level = 16
df = df[ df['training_level'].isin([training_level]) ]

df = df[  df['LED_trial'] == 0 ]

# 1 is right , -1 is left
df['choice'] = df['response_poke'].apply(lambda x: 1 if x == 3 else (-1 if x == 2 else random.choice([1, -1])))

# 1 or 0 if the choice was correct or not
df['correct'] = (df['ILD'] * df['choice']).apply(lambda x: 1 if x > 0 else 0)

# find ABL and ILD
ABL_arr = df['ABL'].unique()
ILD_arr = df['ILD'].unique()


# sort ILD arr in ascending order
ILD_arr = np.sort(ILD_arr)
ABL_arr = np.sort(ABL_arr)

print('ABL:', ABL_arr)
print('ILD:', ILD_arr)

ABL: [20 40 60]
ILD: [-16.  -8.  -4.  -2.  -1.   1.   2.   4.   8.  16.]


In [11]:
# LED off rows
df_led_off = df[ df['LED_trial'] == 0 ]
print(f'len df_led_off = {len(df_led_off)}')

# > 0 and < 1s valid rt 
df_led_off_valid = df_led_off[
    (df_led_off['timed_fix'] - df_led_off['intended_fix'] > 0) &
    (df_led_off['timed_fix'] - df_led_off['intended_fix'] < 1)
]

df_led_off_valid = df_led_off_valid[df_led_off_valid['response_poke'].isin([2,3])]

print(len(df_led_off_valid))

len df_led_off = 66226
52799


In [12]:
V_A = 1.596
theta_A = 2.531
t_A_aff = -0.226
del_go = 0.13

# vbmc

In [ ]:
# up_or_down_RTs_fit_fn(t, V_A, theta_A, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_stim, t_A_aff, t_E_aff, del_go, rate_norm_l, bound, K_max)
# CDF_E_minus_small_t_NORM_rate_norm_fn(t, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, bound, rate_norm_l, K_max) 

In [10]:
def cum_psiam_tied_fn(t, V_A, theta_A, t_A_aff, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_stim, t_E_aff, rate_norm_l, K_max):
    c_A  = cum_A_t_fn(t - t_A_aff, V_A, theta_A)
    c_E = cum_E_t_fn(t - t_E_aff - t_stim, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, rate_norm_l, K_max)
    return c_A + c_E - (c_A * c_E)

In [ ]:
def compute_loglike(row, rate_lambda, T_0, theta_E, Z_E, t_E_aff, rate_norm_l):
    timed_fix = row['timed_fix']
    intended_fix = row['intended_fix']
    
    ILD = row['ILD']
    ABL = row['ABL']
    choice = 2*row['response_poke'] - 5

    rt = timed_fix
    t_stim = intended_fix
    
    K_max = 10
    cum_1 = cum_psiam_tied_fn(t_stim, V_A, theta_A, t_A_aff, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_stim, t_E_aff, rate_norm_l, K_max) 
    cum_2 = cum_psiam_tied_fn(t_stim + 1, V_A, theta_A, t_A_aff, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_stim, t_E_aff, rate_norm_l, K_max)
    trunc_factor = cum_2 - cum_1 + 1e-30

    likelihood = up_or_down_RTs_fit_fn(t, V_A, theta_A, ABL, ILD, rate_lambda, T_0, theta_E, Z_E, t_stim, t_A_aff, t_E_aff, del_go, rate_norm_l, bound, K_max)
    
    


def loglike_fn(params):
    rate_lambda, T_0, theta_E, Z_E, t_E_aff, rate_norm_l = params
   all_loglike = Parallel(n_jobs=30)(delayed(compute_loglike)(row, rate_lambda, T_0, theta_E, Z_E, t_E_aff, rate_norm_l)\
                                       for _, row in df_led_off_valid.iterrows() )